In [ ]:
import math
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

import matplotlib.pyplot as plt
from itertools import islice

from torchlensmaker.raytracing import super_refraction, clamped_refraction, ray_point_squared_distance, refraction
from torchlensmaker.optics import FocalPointLoss, ParallelBeamUniform, ParallelBeamRandom, FixedGap, RefractiveSurface, OpticalStack, Anchor
from torchlensmaker.shapes import Parabola, PiecewiseLine, Line, CircularArc, BezierSpline
from torchlensmaker.training import render, optimize

In [ ]:

lens_radius = 60. # lens radius in mm
surface1 = CircularArc(lens_radius, [-200.])
surface2 = Line(lens_radius, [0.0, 1.0, 0.0]) # y=0 line

optics = OpticalStack([
    ParallelBeamRandom(radius=0.9*lens_radius),
    FixedGap(15.),
    
    RefractiveSurface(surface1, (1.0, 1.49), anchors=(Anchor.Center, Anchor.Edge)),
    FixedGap(5.),
    RefractiveSurface(surface2, (1.49, 1.0)),
    
    FixedGap(100.0), # focal length
    FocalPointLoss(),
])


render(optics, num_rays=10)

optimize(
    optics,
    optimizer = optim.Adam(optics.parameters(), lr=1e-3),
    num_rays = 20,
    num_iter = 150
)

render(optics, num_rays=10)



In [ ]:
from torchlensmaker.optics import Lens
from torchlensmaker.export3d import lens_to_part

lens = Lens(optics.stack[2], optics.stack[3], optics.stack[4])

part = lens_to_part(lens)
part